In [37]:
import pandas as pd
from utils.feature_selection import *
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Classification

In [38]:
final_dataset_path = '../dataset/final/whole/'
acc_dataset_path = final_dataset_path + 'PatchTable_Acceleration_Filtered.csv'
gyro_dataset_path = final_dataset_path + 'PatchTable_Gyroscope_Filtered.csv'

In [39]:
acc_dataset_df = pd.read_csv(acc_dataset_path)
gyro_dataset_df = pd.read_csv(gyro_dataset_path)

In [40]:
acc_dataset_df

,Class,IndexHarmonicity_ML,FirstDerivative_Moment3rd_SigComplete_Magnitude,InterQuartileRange_SigComplete_VT,Std_SigComplete_ML,RelativePower_Below5Hz_AP,Moment2ndPSD_Plom_AP,ModeSignal_AP,RelativePower_BelowDomFr_Magnitude,CentroidSpMax_Magnitude
0,0,-1.069330,0.135301,-0.453811,-0.429353,-0.505171,-0.381535,1.425815,-0.676916,-0.092303
1,1,1.681837,1.559217,-0.610753,-0.770064,-0.370701,0.916358,0.790065,0.522940,2.227281
2,0,1.672111,-0.523202,-0.222828,-0.511502,-0.611616,-0.456534,-0.486181,-0.862825,-0.108542
3,1,1.007142,-0.508966,1.372247,-0.479112,-0.001430,-0.451724,-0.956278,0.246958,1.621542
4,0,0.252562,-0.022803,-0.961819,-0.610377,0.919677,-0.451914,0.405790,-0.747132,-0.473631
5,1,-0.906996,0.133564,-0.727572,0.265277,-1.297334,3.074220,-1.794684,-0.479493,-0.277505
6,0,0.808179,-1.688993,-0.694287,0.869764,1.397099,-0.453804,0.156805,-0.893267,-1.897663
7,1,-0.871964,-0.273250,2.507555,-0.696239,-1.279988,-0.087912,-0.757398,0.611405,0.088414
8,0,-0.267925,0.590969,-0.915989,-0.634120,1.787401,-0.454205,-0.637895,-0.781816,-0.526397
9,1,-0.984467,0.252868,-0.179156,-0.832741,-0.573928,-0.451296,1.935459,2.500430,0.002039


In [41]:
acc_target = acc_dataset_df.iloc[:, 0].copy().values
gyro_target = gyro_dataset_df.iloc[:, 0].copy().values
acc_features = acc_dataset_df.iloc[:, 1:].copy().values
gyro_features = gyro_dataset_df.iloc[:, 1:].copy().values

In [42]:
acc_target

array([0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1])

In [43]:
gyro_features

array([[-0.99608864, -1.68061471, -0.04550722, -1.1724484 , -0.44241608,
        -0.81979075, -0.89880798,  0.9649292 ,  1.14056895],
       [ 0.93287132,  2.11163737, -0.07021977, -0.49128046,  0.12397412,
         0.6109313 , -1.2967928 , -0.13256084, -0.17696923],
       [ 1.45940502,  0.08603492,  0.23339549, -0.59279016,  1.57756657,
        -0.46783144,  0.8473339 , -0.59284529, -0.38643853],
       [-1.03851078, -0.39283755, -0.16647376,  0.38101458,  0.54410194,
         0.1252065 , -0.58662649, -1.11103711,  0.06373277],
       [ 0.19672231,  0.17194626,  0.08182637,  0.1924012 ,  0.31890194,
         0.75543788,  2.40650193,  0.9419858 ,  0.7720133 ],
       [-0.2075358 , -0.10966649,  0.80219126,  0.79601264, -0.06886387,
         0.42973331, -0.01834874, -0.79365987, -0.06598875],
       [-0.78647332, -0.16378657, -0.3613237 , -0.60262311, -0.97899651,
        -0.96472756, -0.23544344,  1.99852933, -2.15605421],
       [ 1.11004617,  0.67128232, -1.1964841 ,  0.81637719, -0

### Logistic Regression

In [44]:
# Assuming X and y are your feature matrix and target vector, respectively

# Initialize the logistic regression model
model = SVC(kernel='linear')

# Initialize Leave-One-Out cross-validator
loo = LeaveOneOut()

# Initialize lists to store predictions and true labels
predictions = []
true_labels = []

# Iterate over each train-test split
for train_index, test_index in loo.split(gyro_features):
    # Split data into train and test sets
    X_train, X_test = acc_features[train_index], acc_features[test_index]
    y_train, y_test = acc_target[train_index], acc_target[test_index]
    
    # Fit the model on the training data
    model.fit(X_train, y_train)
    
    # Make predictions on the test data
    y_pred = model.predict(X_test)
    
    # Store predictions and true labels
    predictions.append(y_pred[0])
    true_labels.append(y_test[0])

# Calculate accuracy
accuracy = accuracy_score(true_labels, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.9166666666666666


### Random Forest

### Support Vector Machine 